In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
!unzip -q gdrive/My\ Drive/train.zip
!unzip -q gdrive/My\ Drive/val.zip


unzip:  cannot find or open gdrive/My Drive/test.zip, gdrive/My Drive/test.zip.zip or gdrive/My Drive/test.zip.ZIP.


In [ ]:
# Data manipulation
import numpy as np
# from sklearn.model_selection import train_test_split  # not needed here because the data is already splitted.
from sklearn.metrics import classification_report  # Recall, F1-Score, precission, support, accuracy and more
from sklearn.utils import shuffle  # for shuffling train and test data
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
import PIL


# ML libraries
import tensorflow as tf

from tensorflow import keras  # keras.utils.model_plot to see the model
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential  
from tensorflow.keras.applications import VGG16  # transfer learning, base model. 
from tensorflow.keras.optimizers import Adam  # Adapt momentum. It uses the benefits of RMSprop and Gradient Descent with momentum. Hyper params: learning rate, beta for momentum
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img  # to load images.

# Visualization
import os
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
preprocess_input = tf.keras.applications.vgg16.preprocess_input

In [ ]:
train_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
valid_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

batch_size = [15,25,35,45,55]

train_batches = tf.keras.utils.image_dataset_from_directory('train',image_size=(224,224),batch_size=32)
val_batches = tf.keras.utils.image_dataset_from_directory('val',image_size=(224,224),batch_size=32)


Found 48500 files belonging to 2 classes.
Found 6929 files belonging to 2 classes.


In [ ]:
base =tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))
base.trainable= False
model = tf.keras.Sequential([
    base,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    
    tf.keras.layers.Dense((1), activation='sigmoid')
    # tf.keras.layers.Flatten(),
    # tf.keras.layers.Dense(1,activation='sigmoid')
])

58889256/58889256 [==============================] - 4s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dropout (Dropout)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 512)               12845568  
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513       
                                                                 
Total params: 27,560,769
Trainable params: 12,846,081
No

In [ ]:
loss = keras.losses.BinaryCrossentropy()
optim = keras.optimizers.Adam()
metrics = ['accuracy']
model.compile(optimizer = optim, loss=loss, metrics = metrics)

In [ ]:
epochs=5
hist = model.fit(train_batches, validation_data=val_batches,
          # callbacks=[early_stopping],
          epochs=epochs)

Epoch 1/5
1516/1516 [==============================] - 272s 171ms/step - loss: 0.5474 - accuracy: 0.8027 - val_loss: 0.5059 - val_accuracy: 0.7926
Epoch 2/5
1516/1516 [==============================] - 258s 170ms/step - loss: 0.3376 - accuracy: 0.8517 - val_loss: 0.5153 - val_accuracy: 0.7855
Epoch 3/5
1516/1516 [==============================] - 246s 162ms/step - loss: 0.2893 - accuracy: 0.8723 - val_loss: 0.5284 - val_accuracy: 0.7971
Epoch 4/5
1516/1516 [==============================] - 257s 169ms/step - loss: 0.2536 - accuracy: 0.8871 - val_loss: 0.5918 - val_accuracy: 0.8020
Epoch 5/5
1516/1516 [==============================] - 257s 169ms/step - loss: 0.2279 - accuracy: 0.8991 - val_loss: 0.6023 - val_accuracy: 0.7939


In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
!unzip -q gdrive/My\ Drive/test.zip

In [ ]:

from pathlib import Path
tp = Path('/content/test/')
test_images = list(tp.glob('*'))
test_images = sorted(test_images,key=lambda x:int(x.stem))
test_images = list(map(str,test_images))
list_ds = tf.data.Dataset.from_tensor_slices(test_images)

In [ ]:
def decode_img(file_path):
  img = tf.io.read_file(file_path)
  # Convert the compressed string to a 3D uint8 tensor
  img = tf.io.decode_jpeg(img, channels=3)
  # Resize the image to the desired size
  return tf.image.resize(img, [224,224])

In [ ]:
test_ds = list_ds.map(decode_img)
test_ds = test_ds.batch(64)
preds = []
for img in test_ds.as_numpy_iterator():
  preds.extend(model.predict(img))

2/2 [==============================] - 0s 128ms/step


In [ ]:
out = np.array(preds)
np.min(out),np.max(out)
out
final_preds = np.round(out)
final_preds
np.unique(final_preds)
final = list(final_preds)
final.count(0), final.count(1)

(2848, 5152)

In [ ]:
import pandas as pd
import csv
print(np.arange(6))
Image_name =np.arange(8000)
Predicted_label= final
df=pd.DataFrame(list(zip(Image_name,Predicted_label)), columns=['Image Name','Predicted Label'])
df.to_csv('/content/Sample_Format.csv', index=False)
final_submission=pd.read_csv('/content/Sample_Format.csv')
final_submission.head(50)

[0 1 2 3 4 5]


,Image Name,Predicted Label
0,0,[1.]
1,1,[0.]
2,2,[1.]
3,3,[1.]
4,4,[1.]
5,5,[1.]
6,6,[0.]
7,7,[1.]
8,8,[1.]
9,9,[1.]
